In [7]:
!pip install flask flask_sqlalchemy pymysql


In [8]:
pip show flask


Name: FlaskNote: you may need to restart the kernel to use updated packages.

Version: 3.0.3
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: C:\Users\chenm\anaconda3\Lib\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: Flask-SQLAlchemy


In [11]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from sqlalchemy import CheckConstraint
from werkzeug.security import generate_password_hash, check_password_hash
from sqlalchemy import text

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///restaurant_system.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Student(db.Model):
    __tablename__ = 'student'
    student_id = db.Column(db.String(9), primary_key=True)  # 改為 String(9)
    student_name = db.Column(db.String(100), nullable=False)
    password = db.Column(db.String(100), nullable=False)

    orders = db.relationship('Order', back_populates='student')
    browses = db.relationship('Browse', back_populates='student')
    comments = db.relationship('Comment', back_populates='student')
    carts = db.relationship('Cart', back_populates='student')

    # 強制 student_id 必須是9個數字（允許前面為0）
    __table_args__ = (
        CheckConstraint("student_id GLOB '[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]'", 
                        name='check_student_id_format'),
    )
# --------------------- Restaurant ---------------------
class Restaurant(db.Model):
    __tablename__ = 'restaurant'
    restaurant_id = db.Column(db.Integer, primary_key=True)
    restaurant_name = db.Column(db.String(100), nullable=False)
    password = db.Column(db.String(100), nullable=False)
    restaurant_type = db.Column(db.String(100))  # 西式, 中式, 日式...
    restaurant_price_range = db.Column(db.String(50))  # $, $$, $$$
    address = db.Column(db.String(200))
    opening_hour = db.Column(db.String(100))  # 例如："09:00-21:00"

    # 關聯其他資料表
    orders = db.relationship('Order', back_populates='restaurant')
    browses = db.relationship('Browse', back_populates='restaurant')
    comments = db.relationship('Comment', back_populates='restaurant')
    coupons = db.relationship('Coupon', back_populates='restaurant')
    menus = db.relationship('Menu', back_populates='restaurant')
    ads = db.relationship('Advertisement', back_populates='restaurant')




# --------------------- CartItem ---------------------
class CartItem(db.Model):
    __tablename__ = 'cart_item'

    cart_item_id = db.Column(db.Integer, primary_key=True)
    student_id = db.Column(db.String(9), db.ForeignKey('student.student_id'))
    menu_item_id = db.Column(db.Integer, db.ForeignKey('menu_item.menu_item_id'))
    quantity = db.Column(db.Integer, default=1)

    student = db.relationship('Student', back_populates='cart_items')
    menu_item = db.relationship('MenuItem')


# --------------------- Order ---------------------
class Order(db.Model):
    __tablename__ = 'order'
    order_id = db.Column(db.Integer, primary_key=True)
    order_time = db.Column(db.DateTime, nullable=False)
    order_price = db.Column(db.Float, nullable=False)
    status = db.Column(db.String(50), default='成立')

    student_id = db.Column(db.String(9), db.ForeignKey('student.student_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    coupon_id = db.Column(db.Integer, db.ForeignKey('coupon.coupon_id'), nullable=True)

    student = db.relationship('Student', back_populates='orders')
    restaurant = db.relationship('Restaurant', back_populates='orders')
    coupon = db.relationship('Coupon')
    order_items = db.relationship('OrderItem', back_populates='order')

# --------------------- OrderItem ---------------------
class OrderItem(db.Model):
    __tablename__ = 'order_item'
    order_item_id = db.Column(db.Integer, primary_key=True)
    meal_name = db.Column(db.String(100), nullable=False)
    quantity = db.Column(db.Integer, default=1)
    price = db.Column(db.Float, nullable=False)

    order_id = db.Column(db.Integer, db.ForeignKey('order.order_id'))
    order = db.relationship('Order', back_populates='order_items')


# --------------------- Menu ---------------------
class Menu(db.Model):
    __tablename__ = 'menu'
    menu_id = db.Column(db.Integer, primary_key=True)
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='menus')
    menu_items = db.relationship('MenuItem', back_populates='menu')

# --------------------- MenuItem ---------------------
class MenuItem(db.Model):
    __tablename__ = 'menu_item'
    menu_item_id = db.Column(db.Integer, primary_key=True)
    item_name = db.Column(db.String(100), nullable=False)
    price = db.Column(db.Float, nullable=False)
    meal_name = db.Column(db.String(100))
    menu_id = db.Column(db.Integer, db.ForeignKey('menu.menu_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    menu = db.relationship('Menu', back_populates='menu_items')

# --------------------- Browse ---------------------
class Browse(db.Model):
    __tablename__ = 'browse'
    browse_id = db.Column(db.Integer, primary_key=True)
    browse_time = db.Column(db.DateTime, default=datetime.utcnow)

    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))

    student = db.relationship('Student', back_populates='browses')
    restaurant = db.relationship('Restaurant', back_populates='browses')

# --------------------- Comment ---------------------
class Comment(db.Model):
    __tablename__ = 'comment'

    comment_id = db.Column(db.Integer, primary_key=True)
    comment_level = db.Column(db.Integer)
    comment_text = db.Column(db.Text, nullable=False)
    respond = db.Column(db.Text, nullable=True)

    student_id = db.Column(db.Integer, db.ForeignKey('student.student_id'))
    student_name = db.Column(db.String(100), nullable=False)  

    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))

    student = db.relationship('Student', back_populates='comments')
    restaurant = db.relationship('Restaurant', back_populates='comments')


# --------------------- Coupon ---------------------
class Coupon(db.Model):
    __tablename__ = 'coupon'
    coupon_id = db.Column(db.Integer, primary_key=True)
    time = db.Column(db.DateTime, default=datetime.utcnow)  # 優惠券建立時間或到期時間
    coupon_offer = db.Column(db.String(255))  # 描述文字，例如「滿300折50」、「全館9折」

    # 新增以下兩個欄位來描述優惠券的折扣類型與數值
    discount_type = db.Column(db.String(20), nullable=False)  # 'fixed' 或 'percentage'
    discount_value = db.Column(db.Float, nullable=False)      # 折扣的數值（例如：50或0.9）
    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='coupons')

# --------------------- Advertisement ---------------------
class Advertisement(db.Model):
    __tablename__ = 'advertisement'
    advertisement_id = db.Column(db.Integer, primary_key=True)
    ad_start_time = db.Column(db.DateTime)
    ad_end_time = db.Column(db.DateTime)
    ad_watch_time = db.Column(db.Integer)
    ad_url = db.Column(db.String(255))

    restaurant_id = db.Column(db.Integer, db.ForeignKey('restaurant.restaurant_id'))
    restaurant = db.relationship('Restaurant', back_populates='ads')

# --------------------- 初始化資料庫 ---------------------
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        print("✅ 資料庫與資料表建立完成！")
    app.run(debug=True)



✅ 資料庫與資料表建立完成！
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\chenm\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
#使用 Flask 框架設定和初始化資料庫（SQLAlchemy）的連線
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///restaurant_system.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

In [15]:
#建立學生帳號
@app.route("/student", methods=["POST"])
def create_student():
    data = request.get_json()
    required_fields = ["student_id", "student_name", "password"]

    if not all(field in data for field in required_fields):
        return jsonify({"message": "❌ 缺少必要欄位(student_id, student_name, password)"}), 400

    hashed_password = generate_password_hash(data["password"])

    sql = text("""
        INSERT INTO student (student_id, student_name, password)
        VALUES (:student_id, :student_name, :password)
    """)
    db.session.execute(sql, {
        "student_id": data["student_id"],
        "student_name": data["student_name"],
        "password": hashed_password
    })
    db.session.commit()

    return jsonify({"message": "✅ 學生新增成功"})

#學生登入驗證
@app.route("/student/login", methods=["POST"])
def student_login():
    data = request.get_json()
    student_id = data.get("student_id")
    password = data.get("password")

    if not student_id or not password:
        return jsonify({"message": "❌ 缺少學生ID或密碼"}), 400

    sql = text("SELECT password FROM student WHERE student_id = :id")
    result = db.session.execute(sql, {"id": student_id}).fetchone()

    if result and check_password_hash(result.password, password):
        return jsonify({"message": "✅ 學生登入成功"})
    else:
        return jsonify({"message": "❌ 登入失敗，帳號或密碼錯誤"}), 401


#更新學生password
@app.route("/student/<string:id>", methods=["PUT"])
def update_student_password(id):
    data = request.get_json()
    new_password = data.get("password")

    if not new_password:
        return jsonify({"message": "❌ 缺少新的 password"}), 400

    hashed_password = generate_password_hash(new_password)

    sql = text("UPDATE student SET password = :password WHERE student_id = :id")
    db.session.execute(sql, {"password": hashed_password, "id": id})
    db.session.commit()

    return jsonify({"message": "✅ 學生密碼更新成功"})


@app.route("/student/<int:id>", methods=["DELETE"])
def delete_student(id):
    sql = text("DELETE FROM student WHERE student_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 學生刪除成功"})

In [17]:
#新增餐廳資料
@app.route("/restaurant", methods=["POST"])
def create_restaurant():
    data = request.get_json()
    required_fields = ["restaurant_name", "password"]

    if not all(field in data for field in required_fields):
        return jsonify({"message": "❌ 缺少餐廳名稱或密碼"}), 400

    hashed_password = generate_password_hash(data["password"])

    sql = text("""
        INSERT INTO restaurant (restaurant_name, password, restaurant_type, restaurant_price_range, address, opening_hour)
        VALUES (:restaurant_name, :password, :restaurant_type, :restaurant_price_range, :address, :opening_hour)
    """)

    db.session.execute(sql, {
        "restaurant_name": data["restaurant_name"],
        "password": hashed_password,
        "restaurant_type": data.get("restaurant_type"),
        "restaurant_price_range": data.get("restaurant_price_range"),
        "address": data.get("address"),
        "opening_hour": data.get("opening_hour")
    })
    db.session.commit()

    return jsonify({"message": "✅ 餐廳建立成功"})
#更新密碼
@app.route("/restaurant/<int:id>", methods=["PUT"])
def update_restaurant_password(id):
    data = request.get_json()
    new_password = data.get("password")

    if not new_password:
        return jsonify({"message": "❌ 缺少新的 password"}), 400

    hashed_password = generate_password_hash(new_password)

    sql = text("UPDATE restaurant SET password = :password WHERE restaurant_id = :id")
    db.session.execute(sql, {"password": hashed_password, "id": id})
    db.session.commit()

    return jsonify({"message": "✅ 餐廳密碼更新成功"})
#餐廳方進行登入
@app.route("/restaurant/login", methods=["POST"])
def restaurant_login():
    data = request.get_json()
    restaurant_id = data.get("restaurant_id")
    password = data.get("password")

    if not restaurant_id or not password:
        return jsonify({"message": "❌ 缺少餐廳ID或密碼"}), 400

    sql = text("SELECT password FROM restaurant WHERE restaurant_id = :id")
    result = db.session.execute(sql, {"id": restaurant_id}).fetchone()

    if result and check_password_hash(result.password, password):
        return jsonify({"message": "✅ 登入成功"})
    else:
        return jsonify({"message": "❌ 登入失敗，帳號或密碼錯誤"}), 401
# 刪除餐廳帳號
@app.route("/restaurant/<int:id>", methods=["DELETE"])
def delete_restaurant(id):
    sql = text("DELETE FROM restaurant WHERE restaurant_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 餐廳刪除成功"})
# 取得所有餐廳資訊
@app.route("/restaurants", methods=["GET"])
def get_restaurants():
    sql = text("SELECT restaurant_id, restaurant_name, restaurant_type, restaurant_price_range, address, opening_hour FROM restaurant")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])


In [19]:
#新增菜單API
@app.route("/menu_item", methods=["POST"])
def create_menu_item():
    data = request.get_json()
    item = MenuItem(**data)
    db.session.add(item)
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目新增成功"})
#取得菜單
@app.route("/menu_items", methods=["GET"])
def get_menu_items():
    items = MenuItem.query.all()
    return jsonify([{
        "menu_item_id": item.menu_item_id,
        "item_name": item.item_name,
        "price": item.price,
        "meal_name": item.meal_name
    } for item in items])
#更新菜單
@app.route("/menu_item/<int:menu_item_id>", methods=["PUT"])
def update_menu_item(menu_item_id):
    data = request.get_json()
    item = MenuItem.query.get(menu_item_id)

    if not item:
        return jsonify({"error": "❌ 找不到此菜單項目"}), 404

    # 更新菜單項目的各個欄位
    item.item_name = data.get("item_name", item.item_name)
    item.price = data.get("price", item.price)
    item.meal_name = data.get("meal_name", item.meal_name)
    item.menu_id = data.get("menu_id", item.menu_id)
    item.restaurant_id = data.get("restaurant_id", item.restaurant_id)

    db.session.commit()

    return jsonify({"message": "✅ 菜單項目更新成功"})
#刪除菜單
@app.route("/menu_item/<int:menu_item_id>", methods=["DELETE"])
def delete_menu_item(menu_item_id):
    item = MenuItem.query.get(menu_item_id)

    if not item:
        return jsonify({"error": "❌ 找不到此菜單項目"}), 404

    db.session.delete(item)
    db.session.commit()

    return jsonify({"message": "✅ 菜單項目刪除成功"})



In [21]:
#新增購物車
@app.route("/cart_item", methods=["POST"])
def create_cart_item():
    data = request.get_json()
    sql = text("SELECT price FROM menu_item WHERE menu_item_id = :id")
    result = db.session.execute(sql, {"id": data["menu_item_id"]}).fetchone()

    if not result:
        return jsonify({"error": "❌ 找不到餐點"}), 404

    price = result.price

    # 插入購物車資料（純SQL）
    insert_sql = text("""
        INSERT INTO cart_item (student_id, menu_item_id, quantity, price)
        VALUES (:student_id, :menu_item_id, :quantity, :price)
    """)
    db.session.execute(insert_sql, {
        "student_id": data["student_id"],
        "menu_item_id": data["menu_item_id"],
        "quantity": data.get("quantity", 1),
        "price": price
    })
    db.session.commit()

    return jsonify({"message": "✅ 加入購物車成功"})


#查看購物車內容
@app.route("/cart_items/<string:student_id>", methods=["GET"])
def get_cart_items(student_id):
    sql = text("""
        SELECT ci.cart_item_id, mi.item_name, ci.price, ci.quantity
        FROM cart_item ci
        JOIN menu_item mi ON ci.menu_item_id = mi.menu_item_id
        WHERE ci.student_id = :student_id
    """)

    result = db.session.execute(sql, {"student_id": student_id}).fetchall()

    items = [{
        "cart_item_id": row.cart_item_id,
        "item_name": row.item_name,
        "price": row.price,
        "quantity": row.quantity
    } for row in result]

    return jsonify(items)

# 更新購物車項目數量
@app.route("/cart_item/<int:cart_item_id>", methods=["PUT"])
def update_cart_item(cart_item_id):
    data = request.get_json()

    sql = text("""
        UPDATE cart_item
        SET quantity = :quantity
        WHERE cart_item_id = :id
    """)
    result = db.session.execute(sql, {
        "quantity": data.get("quantity"),
        "id": cart_item_id
    })

    if result.rowcount == 0:
        return jsonify({"error": "❌ 找不到此購物車項目"}), 404

    db.session.commit()

    return jsonify({"message": "✅ 購物車數量更新成功"})



#付款的API
@app.route("/checkout/<string:student_id>", methods=["POST"])
def checkout(student_id):
    data = request.get_json()
    coupon_id = data.get("coupon_id")

    cart_items = CartItem.query.filter_by(student_id=student_id).all()
    if not cart_items:
        return jsonify({"error": "❌ 購物車為空"}), 400

    total_price = sum(item.quantity * item.price for item in cart_items)

    discount_amount = 0
    if coupon_id:
        coupon = Coupon.query.get(coupon_id)
        if coupon.discount_type == 'fixed':
            discount_amount = coupon.discount_value
        elif coupon.discount_type == 'percentage':
            discount_amount = total_price * (1 - coupon.discount_value)

    final_price = max(total_price - discount_amount, 0)

    # 建立訂單（純SQL）
    order_sql = text("""
        INSERT INTO "order" (order_time, order_price, student_id, restaurant_id, coupon_id, status)
        VALUES (:order_time, :order_price, :student_id, :restaurant_id, :coupon_id, :status)
        RETURNING order_id
    """)

    result = db.session.execute(order_sql, {
        "order_time": datetime.utcnow(),
        "order_price": final_price,
        "student_id": student_id,
        "restaurant_id": cart_items[0].menu_item.restaurant_id,
        "coupon_id": coupon_id,
        "status": "已付款"
    })
    new_order_id = result.fetchone().order_id

    # 訂單明細（純SQL）
    for item in cart_items:
        order_item_sql = text("""
            INSERT INTO order_item (meal_name, quantity, price, order_id)
            VALUES (:meal_name, :quantity, :price, :order_id)
        """)
        db.session.execute(order_item_sql, {
            "meal_name": item.menu_item.item_name,
            "quantity": item.quantity,
            "price": item.price,
            "order_id": new_order_id
        })

    # 清空購物車（純SQL）
    delete_sql = text("DELETE FROM cart_item WHERE student_id = :student_id")
    db.session.execute(delete_sql, {"student_id": student_id})

    db.session.commit()

    return jsonify({
        "message": "✅ 訂單建立並付款成功 (Demo)",
        "order_id": new_order_id,
        "original_price": total_price,
        "final_price": final_price,
        "discount_amount": discount_amount
    })


In [ ]:
#新增留言
@app.route("/comment", methods=["POST"])
def create_comment():
    data = request.get_json()

    # 透過學生ID 查詢學生名字
    student = Student.query.get(data["student_id"])
    if not student:
        return jsonify({"error": "❌ 找不到該學生"}), 404

    new_comment = Comment(
        comment_level=data.get("comment_level"),
        comment_text=data["comment_text"],
        respond=data.get("respond"),
        student_id=student.student_id,
        student_name=student.student_name,  # 存入留言者的名字
        restaurant_id=data["restaurant_id"]
    )

    db.session.add(new_comment)
    db.session.commit()

    return jsonify({"message": "✅ 留言新增成功"})
#取得留言
@app.route("/comments", methods=["GET"])
def get_comments():
    comments = Comment.query.all()
    return jsonify([{
        "comment_id": c.comment_id,
        "comment_level": c.comment_level,
        "comment_text": c.comment_text,
        "respond": c.respond,
        "student_id": c.student_id,
        "student_name": c.student_name,
        "restaurant_id": c.restaurant_id,
        "restaurant_name": c.restaurant.restaurant_name
    } for c in comments])

#更新留言
@app.route("/comment/<int:comment_id>", methods=["PUT"])
def update_comment(comment_id):
    data = request.get_json()
    c = Comment.query.get(comment_id)
    if not c:
        return jsonify({"error": "❌ 找不到該留言"}), 404

    c.comment_level = data.get("comment_level", c.comment_level)
    c.comment_text = data.get("comment_text", c.comment_text)
    c.respond = data.get("respond", c.respond)

    db.session.commit()
    return jsonify({"message": "✅ 留言更新成功"})
#刪除留言
@app.route("/comment/<int:comment_id>", methods=["DELETE"])
def delete_comment(comment_id):
    c = Comment.query.get(comment_id)
    if not c:
        return jsonify({"error": "❌ 找不到該留言"}), 404

    db.session.delete(c)
    db.session.commit()
    return jsonify({"message": "✅ 留言刪除成功"})


In [1]:

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///restaurant_system.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# --------------------- Cart CRUD ---------------------
@app.route("/cart", methods=["POST"])
def create_cart():
    data = request.get_json()
    sql = text("""
        INSERT INTO cart (student_id)
        VALUES (:student_id)
    """)
    db.session.execute(sql, {"student_id": data.get("student_id")})
    db.session.commit()
    return jsonify({"message": "✅ 購物車新增成功"})

@app.route("/carts", methods=["GET"])
def get_all_carts():
    sql = text("SELECT * FROM cart")
    result = db.session.execute(sql)
    carts = [dict(row) for row in result.mappings()]
    return jsonify(carts)
def update_cart(cart_id):
    data = request.get_json()
    sql = text("UPDATE cart SET student_id = :student_id WHERE cart_id = :id")
    db.session.execute(sql, {"student_id": data.get("student_id"), "id": cart_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車已更新"})
    
@app.route("/cart/<int:cart_id>", methods=["DELETE"])
def delete_cart(cart_id):
    sql = text("DELETE FROM cart WHERE cart_id = :id")
    db.session.execute(sql, {"id": cart_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車已刪除"})

# --------------------- CartItem CRUD ---------------------
@app.route("/cart_item", methods=["POST"])
def create_cart_item():
    data = request.get_json()
    sql = text("""
        INSERT INTO cart_item (cart_id, menu_item_id, quantity)
        VALUES (:cart_id, :menu_item_id, :quantity)
    """)
    db.session.execute(sql, {
        "cart_id": data.get("cart_id"),
        "menu_item_id": data.get("menu_item_id"),
        "quantity": data.get("quantity")
    })
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目新增成功"})

@app.route("/cart_items", methods=["GET"])
def get_all_cart_items():
    sql = text("SELECT * FROM cart_item")
    result = db.session.execute(sql)
    items = [dict(row) for row in result.mappings()]
    return jsonify(items)

@app.route("/cart_item/<int:cart_item_id>", methods=["PUT"])
def update_cart_item(cart_item_id):
    data = request.get_json()
    sql = text("""
        UPDATE cart_item
        SET cart_id = :cart_id,
            menu_item_id = :menu_item_id,
            quantity = :quantity
        WHERE cart_item_id = :id
    """)
    db.session.execute(sql, {
        "cart_id": data.get("cart_id"),
        "menu_item_id": data.get("menu_item_id"),
        "quantity": data.get("quantity"),
        "id": cart_item_id
    })
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目已更新"})

@app.route("/cart_item/<int:cart_item_id>", methods=["DELETE"])
def delete_cart_item(cart_item_id):
    sql = text("DELETE FROM cart_item WHERE cart_item_id = :id")
    db.session.execute(sql, {"id": cart_item_id})
    db.session.commit()
    return jsonify({"message": "✅ 購物車項目已刪除"})

# --------------------- MenuItem CRUD ---------------------
@app.route("/menu_item", methods=["POST"])
def create_menu_item():
    data = request.get_json()
    sql = text("""
        INSERT INTO menu_item (item_name, price, meal_name, menu_id, restaurant_id)
        VALUES (:item_name, :price, :meal_name, :menu_id, :restaurant_id)
    """)
    db.session.execute(sql, {
        "item_name": data.get("item_name"),
        "price": data.get("price"),
        "meal_name": data.get("meal_name"),
        "menu_id": data.get("menu_id"),
        "restaurant_id": data.get("restaurant_id")
    })
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目新增成功"})

@app.route("/menu_items", methods=["GET"])
def get_all_menu_items():
    sql = text("SELECT * FROM menu_item")
    result = db.session.execute(sql)
    items = [dict(row) for row in result.mappings()]
    return jsonify(items)

@app.route("/menu_item/<int:menu_item_id>", methods=["PUT"])
def update_menu_item(menu_item_id):
    data = request.get_json()
    sql = text("""
        UPDATE menu_item
        SET item_name = :item_name,
            price = :price,
            meal_name = :meal_name,
            menu_id = :menu_id,
            restaurant_id = :restaurant_id
        WHERE menu_item_id = :id
    """)
    db.session.execute(sql, {
        "item_name": data.get("item_name"),
        "price": data.get("price"),
        "meal_name": data.get("meal_name"),
        "menu_id": data.get("menu_id"),
        "restaurant_id": data.get("restaurant_id"),
        "id": menu_item_id
    })
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目已更新"})

@app.route("/menu_item/<int:menu_item_id>", methods=["DELETE"])
def delete_menu_item(menu_item_id):
    sql = text("DELETE FROM menu_item WHERE menu_item_id = :id")
    db.session.execute(sql, {"id": menu_item_id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單項目已刪除"})
# --------------------- Menu CRUD ---------------------
@app.route("/menu", methods=["POST"])
def create_menu():
    data = request.get_json()
    sql = text("INSERT INTO menu (average_price, restaurant_id) VALUES (:average_price, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 菜單新增成功"})

@app.route("/menus", methods=["GET"])
def get_menus():
    sql = text("SELECT * FROM menu")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/menu/<int:id>", methods=["GET"])
def get_menu(id):
    sql = text("SELECT * FROM menu WHERE menu_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到菜單"}), 404)

@app.route("/menu/<int:id>", methods=["PUT"])
def update_menu(id):
    data = request.get_json()
    sql = text("UPDATE menu SET average_price = :average_price, restaurant_id = :restaurant_id WHERE menu_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單更新成功"})

@app.route("/menu/<int:id>", methods=["DELETE"])
def delete_menu(id):
    sql = text("DELETE FROM menu WHERE menu_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 菜單刪除成功"})

# --------------------- Browse CRUD ---------------------
@app.route("/browse", methods=["POST"])
def create_browse():
    data = request.get_json()
    sql = text("INSERT INTO browse (browse_time, student_id, restaurant_id) VALUES (:browse_time, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 瀏覽紀錄新增成功"})

@app.route("/browses", methods=["GET"])
def get_browses():
    sql = text("SELECT * FROM browse")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/browse/<int:id>", methods=["GET"])
def get_browse(id):
    sql = text("SELECT * FROM browse WHERE browse_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到瀏覽紀錄"}), 404)

@app.route("/browse/<int:id>", methods=["DELETE"])
def delete_browse(id):
    sql = text("DELETE FROM browse WHERE browse_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 瀏覽紀錄刪除成功"})

# --------------------- Comment CRUD ---------------------
@app.route("/comment", methods=["POST"])
def create_comment():
    data = request.get_json()
    sql = text("INSERT INTO comment (comment_level, comment_text, respond, parent_id, student_id, restaurant_id) VALUES (:comment_level, :comment_text, :respond, :parent_id, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 留言新增成功"})

@app.route("/comments", methods=["GET"])
def get_comments():
    sql = text("SELECT * FROM comment")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/comment/<int:id>", methods=["GET"])
def get_comment(id):
    sql = text("SELECT * FROM comment WHERE comment_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到留言"}), 404)

@app.route("/comment/<int:id>", methods=["PUT"])
def update_comment(id):
    data = request.get_json()
    sql = text("UPDATE comment SET comment_level = :comment_level, comment_text = :comment_text, respond = :respond, parent_id = :parent_id, student_id = :student_id, restaurant_id = :restaurant_id WHERE comment_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 留言更新成功"})

@app.route("/comment/<int:id>", methods=["DELETE"])
def delete_comment(id):
    sql = text("DELETE FROM comment WHERE comment_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 留言刪除成功"})

# --------------------- Coupon CRUD ---------------------
@app.route("/coupon", methods=["POST"])
def create_coupon():
    data = request.get_json()
    sql = text("INSERT INTO coupon (time, restaurant_id) VALUES (:time, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 優惠券新增成功"})

@app.route("/coupons", methods=["GET"])
def get_coupons():
    sql = text("SELECT * FROM coupon")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/coupon/<int:id>", methods=["GET"])
def get_coupon(id):
    sql = text("SELECT * FROM coupon WHERE coupon_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到優惠券"}), 404)

@app.route("/coupon/<int:id>", methods=["PUT"])
def update_coupon(id):
    data = request.get_json()
    sql = text("UPDATE coupon SET time = :time, restaurant_id = :restaurant_id WHERE coupon_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 優惠券更新成功"})

@app.route("/coupon/<int:id>", methods=["DELETE"])
def delete_coupon(id):
    sql = text("DELETE FROM coupon WHERE coupon_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 優惠券刪除成功"})

# --------------------- Advertisement CRUD ---------------------
@app.route("/advertisement", methods=["POST"])
def create_advertisement():
    data = request.get_json()
    sql = text("INSERT INTO advertisement (ad_start_time, ad_end_time, ad_watch_time, ad_url, restaurant_id) VALUES (:ad_start_time, :ad_end_time, :ad_watch_time, :ad_url, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 廣告新增成功"})

@app.route("/advertisements", methods=["GET"])
def get_advertisements():
    sql = text("SELECT * FROM advertisement")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/advertisement/<int:id>", methods=["GET"])
def get_advertisement(id):
    sql = text("SELECT * FROM advertisement WHERE advertisement_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到廣告"}), 404)

@app.route("/advertisement/<int:id>", methods=["PUT"])
def update_advertisement(id):
    data = request.get_json()
    sql = text("UPDATE advertisement SET ad_start_time = :ad_start_time, ad_end_time = :ad_end_time, ad_watch_time = :ad_watch_time, ad_url = :ad_url, restaurant_id = :restaurant_id WHERE advertisement_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 廣告更新成功"})

@app.route("/advertisement/<int:id>", methods=["DELETE"])
def delete_advertisement(id):
    sql = text("DELETE FROM advertisement WHERE advertisement_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 廣告刪除成功"})



# --------------------- Order CRUD ---------------------
@app.route("/order", methods=["POST"])
def create_order():
    data = request.get_json()
    sql = text("INSERT INTO 'order' (order_time, order_price, student_id, restaurant_id) VALUES (:order_time, :order_price, :student_id, :restaurant_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 訂單新增成功"})

@app.route("/orders", methods=["GET"])
def get_orders():
    sql = text("SELECT * FROM 'order'")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/order/<int:id>", methods=["GET"])
def get_order(id):
    sql = text("SELECT * FROM 'order' WHERE order_id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到訂單"}), 404)

@app.route("/order/<int:id>", methods=["PUT"])
def update_order(id):
    data = request.get_json()
    sql = text("UPDATE 'order' SET order_time = :order_time, order_price = :order_price, student_id = :student_id, restaurant_id = :restaurant_id WHERE order_id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單更新成功"})

@app.route("/order/<int:id>", methods=["DELETE"])
def delete_order(id):
    sql = text("DELETE FROM 'order' WHERE order_id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單刪除成功"})

# --------------------- OrderItem CRUD ---------------------
@app.route("/order_item", methods=["POST"])
def create_order_item():
    data = request.get_json()
    sql = text("INSERT INTO order_item (meal_name, quantity, price, order_id) VALUES (:meal_name, :quantity, :price, :order_id)")
    db.session.execute(sql, data)
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細新增成功"})

@app.route("/order_items", methods=["GET"])
def get_order_items():
    sql = text("SELECT * FROM order_item")
    result = db.session.execute(sql)
    return jsonify([dict(row) for row in result.mappings()])

@app.route("/order_item/<int:id>", methods=["GET"])
def get_order_item(id):
    sql = text("SELECT * FROM order_item WHERE id = :id")
    result = db.session.execute(sql, {"id": id}).mappings().first()
    return jsonify(dict(result)) if result else (jsonify({"error": "找不到訂單明細"}), 404)

@app.route("/order_item/<int:id>", methods=["PUT"])
def update_order_item(id):
    data = request.get_json()
    sql = text("UPDATE order_item SET meal_name = :meal_name, quantity = :quantity, price = :price, order_id = :order_id WHERE id = :id")
    db.session.execute(sql, {**data, "id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細更新成功"})

@app.route("/order_item/<int:id>", methods=["DELETE"])
def delete_order_item(id):
    sql = text("DELETE FROM order_item WHERE id = :id")
    db.session.execute(sql, {"id": id})
    db.session.commit()
    return jsonify({"message": "✅ 訂單明細刪除成功"})

#讓average price跟 menuitem price變更
def update_average_price(menu_id, session):
    avg_price = session.query(func.avg(MenuItem.price)).filter(MenuItem.menu_id == menu_id).scalar()
    menu = session.query(Menu).get(menu_id)
    menu.average_price = avg_price if avg_price else 0
    session.commit()

# 設定事件監聽器
@event.listens_for(MenuItem, 'after_insert')
def after_insert(mapper, connection, target):
    session = object_session(target)
    update_average_price(target.menu_id, session)

@event.listens_for(MenuItem, 'after_update')
def after_update(mapper, connection, target):
    session = object_session(target)
    update_average_price(target.menu_id, session)

@event.listens_for(MenuItem, 'after_delete')
def after_delete(mapper, connection, target):
    session = object_session(target)
    update_average_price(target.menu_id, session)



# --------------------- 初始化資料庫 ---------------------
if __name__ == '__main__':
    with app.app_context():
        db.create_all()
        print("✅ 資料庫與資料表建立完成！")
    app.run(debug=True)



✅ 資料庫與資料表建立完成！
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\chenm\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
